# Open Space on Department of Education Sites

## Data Sources

- MapPLUTO through NYC Digital Twin
- [Building Footprints](https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh)
[docs](https://github.com/CityOfNewYork/nyc-geo-metadata/blob/master/Metadata/Metadata_BuildingFootprints.md)
- [Land Cover Raster](https://data.cityofnewyork.us/Environment/Land-Cover-Raster-Data-2017-6in-Resolution/he6d-2qns)
[docs](https://github.com/CityOfNewYork/nyc-geo-metadata/blob/master/Metadata/Metadata_LandCover.md)
- [Elevation Raster]() 


In [19]:
import os
import geopandas as gpd
from dotenv import load_dotenv, find_dotenv

from geoalchemy2 import Geometry, WKTElement
import ipyleaflet
import pandas as pd
import psycopg2
import pyproj
from rasterio.io import MemoryFile
from shapely.geometry import shape
from shapely.ops import transform
import sqlalchemy

In [2]:
pd.set_option("display.precision", 8)

In [3]:
# load environment variables from .env file
load_dotenv(find_dotenv())

True

In [4]:
# tmp folder for downloaded data
LOCAL_PATH = os.getenv("LOCAL_PATH")

In [8]:
# remote db connection
conn_vars = ['REMOTE_TMP_USER', 'REMOTE_TMP_PASSWORD', 'REMOTE_TMP_HOST', 'REMOTE_TMP_PORT', 'REMOTE_TMP_DB']
user, password, host, port, dbname = [os.getenv(var) for var in conn_vars]
conn_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
engine = sqlalchemy.create_engine(conn_string)

In [23]:
# local db connection
local_conn_vars = ['DEV_USER', 'DEV_PASSWORD', 'DEV_HOST', 'DEV_PORT', 'DEV_DB']
luser, lpassword, lhost, lport, ldbname = [os.getenv(var) for var in conn_vars]
local_conn_string = f'postgresql://{luser}:{lpassword}@{lhost}:{lport}/{ldbname}'
local_conn = psycopg2.connect(local_conn_string)


## Open Space on DOE Lots

In [10]:
with open('../queries/2020-07-21_doe-open-space.sql', 'r') as file:
    f = file.read()
    openspace_query = sqlalchemy.text(f)

In [11]:
# get geodataframe from postgis
openspace_gdf = gpd.read_postgis(openspace_query, engine, geom_col='geometry')

In [13]:
openspace_gdf

,geometry,bbl,ownername
0,"POLYGON ((980585.862 200939.528, 980563.536 20...",1000160215.0,NYC DEPARTMENT OF EDUCATION
1,"MULTIPOLYGON (((985184.455 198016.292, 984775....",1001110160.0,NYC DEPARTMENT OF EDUCATION
2,None,1001420001.0,NYC DEPARTMENT OF EDUCATION
3,None,1002360006.0,NYC DEPARTMENT OF EDUCATION
4,"POLYGON ((984805.422 201174.561, 984795.074 20...",1002360038.0,NYC DEPARTMENT OF EDUCATION
...,...,...,...
1095,"POLYGON ((923415.126 136802.876, 923420.950 13...",5070920039.0,NYC DEPARTMENT OF EDUCATION
1096,"POLYGON ((918243.790 136113.764, 918695.519 13...",5073150007.0,NYC DEPARTMENT OF EDUCATION
1097,"POLYGON ((920502.139 124994.901, 920171.105 12...",5077600001.0,NYC DEPARTMENT OF EDUCATION
1098,"POLYGON ((916288.700 124518.554, 916594.881 12...",5078940001.0,NYC DEPARTMENT OF EDUCATION


In [46]:
openspace_shapes = openspace_gdf['geometry'].dropna().apply(lambda g: g.wkt).tolist()
openspace_shapes[0]

'POLYGON ((980585.8616470844 200939.5279607624, 980563.5362324268 200823.3667757511, 980550.8800897598 200757.518482253, 980536.9132541716 200684.8486800939, 980195.9920200109 200888.76231426, 980261.2327031791 200997.9609147608, 980585.8616470844 200939.5279607624), (980214.2774165596 200888.5415141637, 980268.872451723 200855.9428260863, 980445.5771508076 200750.4318822374, 980475.3385742275 200732.6612484974, 980533.4149016454 200697.9838244066, 980539.0713863927 200728.3446560857, 980542.2285323148 200745.2904883227, 980552.8528548919 200802.3146524962, 980553.0083663916 200803.5318416544, 980553.0415028206 200803.8622215706, 980553.0575788927 200804.0321687495, 980553.0798885694 200804.2933230749, 980553.1025263077 200804.5902385021, 980553.1422244027 200805.2523106502, 980553.1546915626 200805.5354465732, 980553.1694553039 200805.995091318, 980553.178313569 200806.6748799869, 980553.174704641 200807.1768474951, 980553.1527230564 200807.9609666617, 980553.1110564786 200808.7273693

## Slope & Landcover

In [42]:
with open('../queries/2020-07-23_doe-open-space-slope-landcover.sql', 'r') as file:
    f = file.read()
    landscape_query = f

In [43]:
landscape_query

'SELECT\n  ST_AsTIFF(\n    ST_Clip(\n      lc.rast,\n      ST_Collect(%(bounds)s)\n    )\n  )\nFROM\n  doitt_landcover_v2017 lc\n;'

In [44]:
# get raster data from postgis
# landscape_gdf = gpd.read_postgis(landscape_query, engine, geom_col='geometry', params={'bounds': openspace_shapes})
local_cursor = local_conn.cursor()
local_cursor.execute(landscape_query, {'bounds': openspace_shapes})

for row in local_cursor:
    rast = row[0].tobytes()
    with MemoryFile(rast).open() as dataset:
        data_array = dataset.read()
        print(data_array)

    # data = rs.fetchone()[0]
    # print("Data: %s" % data)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [10]:
landscape_gdf

,geometry,bbl,ownername
0,"POLYGON ((980585.862 200939.528, 980563.536 20...",1000160215.0,NYC DEPARTMENT OF EDUCATION


## Misc

In [56]:
openspace_gdf.to_file("/Users/carstenrodin/Desktop/tmp/openspace3.geojson", driver='GeoJSON')

In [49]:
with open('../queries/2020-07-21_parks.sql', 'r') as file:
    f = file.read()
    parks_query = sqlalchemy.text(f)

In [50]:
parks_gdf = gpd.read_postgis(parks_query, engine, geom_col='geometry')

In [51]:
parks_gdf.to_file("/Users/carstenrodin/Desktop/tmp/parks.geojson", driver='GeoJSON')